<a href="https://colab.research.google.com/github/afullhart/climateanalogs/blob/main/Figures/3D_Clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f

In [ ]:
!pip install rioxarray
!pip install torchmetrics
!pip install permetrics

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rioxarray as rx
import os
